In [1]:
# Import necessary libraries
import os
import sys
import pandas as pd
# Add the scripts folder to the Python path
sys.path.append(os.path.abspath("../scripts"))

In [2]:
# Import the load_data function from the data_loader module
from data_loader import load_data
from data_preprocessing import convert_data_types

In [3]:

# Define file path
file_path = 'D:/Kifya_training/Week 3/Week 3/AlphaCare-Insurance-Analysis/notebooks/data/MachineLearningRating_v3.txt'

# Load data using the custom function
data = pd.read_csv(file_path, delimiter='|')

# Display the first few rows of the data
print(data.head())

C:\Users\Yoni\AppData\Local\Temp\ipykernel_2400\317629331.py:5: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, delimiter='|')


   UnderwrittenCoverID  PolicyID     TransactionMonth  IsVATRegistered  \
0               145249     12827  2015-03-01 00:00:00             True   
1               145249     12827  2015-05-01 00:00:00             True   
2               145249     12827  2015-07-01 00:00:00             True   
3               145255     12827  2015-05-01 00:00:00             True   
4               145255     12827  2015-07-01 00:00:00             True   

  Citizenship          LegalType Title Language                 Bank  \
0              Close Corporation    Mr  English  First National Bank   
1              Close Corporation    Mr  English  First National Bank   
2              Close Corporation    Mr  English  First National Bank   
3              Close Corporation    Mr  English  First National Bank   
4              Close Corporation    Mr  English  First National Bank   

       AccountType  ...                    ExcessSelected CoverCategory  \
0  Current account  ...             Mobility - 

In [4]:
# Check the data types
print(data.dtypes)

UnderwrittenCoverID           int64
PolicyID                      int64
TransactionMonth             object
IsVATRegistered                bool
Citizenship                  object
LegalType                    object
Title                        object
Language                     object
Bank                         object
AccountType                  object
MaritalStatus                object
Gender                       object
Country                      object
Province                     object
PostalCode                    int64
MainCrestaZone               object
SubCrestaZone                object
ItemType                     object
mmcode                      float64
VehicleType                  object
RegistrationYear              int64
make                         object
Model                        object
Cylinders                   float64
cubiccapacity               float64
kilowatts                   float64
bodytype                     object
NumberOfDoors               

In [5]:
# Define the column types to convert to
column_types = {
    'TransactionMonth': 'datetime64[ns]',
    'VehicleIntroDate': 'datetime64[ns]',
    'CapitalOutstanding': 'category',  # If the column contains specific categories, use category
    'NewVehicle': 'category', 
    'WrittenOff': 'category', 
    'Rebuilt': 'category', 
    'Converted': 'category', 
    'CrossBorder': 'category', 
    'Cylinders': 'float64',
    'cubiccapacity': 'float64',
    'kilowatts': 'float64',
    'NumberOfDoors': 'float64',
    'NumberOfVehiclesInFleet': 'float64',
    'SumInsured': 'float64',
    'CalculatedPremiumPerTerm': 'float64',
    'TotalPremium': 'float64',
    'TotalClaims': 'float64',
    'PostalCode': 'int64',
    'PolicyID': 'int64',
    'UnderwrittenCoverID': 'int64',
}

# Apply the conversion
data = convert_data_types(data, column_types)

# Check the updated data types
print(data.dtypes)

UnderwrittenCoverID                  int64
PolicyID                             int64
TransactionMonth            datetime64[ns]
IsVATRegistered                       bool
Citizenship                         object
LegalType                           object
Title                               object
Language                            object
Bank                                object
AccountType                         object
MaritalStatus                       object
Gender                              object
Country                             object
Province                            object
PostalCode                           int64
MainCrestaZone                      object
SubCrestaZone                       object
ItemType                            object
mmcode                             float64
VehicleType                         object
RegistrationYear                     int64
make                                object
Model                               object
Cylinders  

In [6]:
# Check for missing values in the dataset
missing_values = data.isnull().sum()

# Display columns with missing values
missing_values[missing_values > 0]


Bank                        145961
AccountType                  40232
MaritalStatus                 8259
Gender                        9536
mmcode                         552
VehicleType                    552
make                           552
Model                          552
Cylinders                      552
cubiccapacity                  552
kilowatts                      552
bodytype                       552
NumberOfDoors                  552
VehicleIntroDate               552
CustomValueEstimate         779642
CapitalOutstanding               2
NewVehicle                  153295
WrittenOff                  641901
Rebuilt                     641901
Converted                   641901
CrossBorder                 999400
NumberOfVehiclesInFleet    1000098
dtype: int64

In [10]:
# Handle Missing Values
# Fill categorical columns with "Not specified"
categorical_columns = ['Bank', 'AccountType', 'MaritalStatus', 'Gender', 
                       'VehicleType', 'make', 'Model', 'bodytype', 'VehicleIntroDate', 
                       'AlarmImmobiliser', 'TrackingDevice', 'NewVehicle', 'WrittenOff', 
                       'Rebuilt', 'Converted', 'CrossBorder', 'CoverCategory', 
                       'CoverType', 'CoverGroup', 'Section', 'Product', 'StatutoryClass', 
                       'StatutoryRiskType']

for col in categorical_columns:
    if data[col].dtype.name == 'category':  # Handle categorical columns
        if "Not specified" not in data[col].cat.categories:
            data[col] = data[col].cat.add_categories(["Not specified"])
        data[col] = data[col].fillna("Not specified")
    elif data[col].dtype.name == 'datetime64[ns]':  # Handle datetime columns
        data[col] = data[col].fillna(pd.NaT)  # Fill with NaT for missing datetime
    else:  # Handle other object-type columns
        data[col] = data[col].fillna("Not specified")


In [12]:
# Define numerical columns
numerical_columns = ['Cylinders', 'cubiccapacity', 'kilowatts', 'NumberOfDoors', 
                     'CustomValueEstimate', 'CapitalOutstanding', 'NumberOfVehiclesInFleet']

# Define columns for grouping (same as the key columns you use for matching)
grouping_columns = ['VehicleType', 'make', 'Model']

# Ensure there are no missing values in the grouping columns# Define numerical columns
numerical_columns = ['Cylinders', 'cubiccapacity', 'kilowatts', 'NumberOfDoors', 
                     'CustomValueEstimate', 'CapitalOutstanding', 'NumberOfVehiclesInFleet']

# Define columns for grouping (same as the key columns you use for matching)
grouping_columns = ['VehicleType', 'make', 'Model']

# Ensure the grouping columns are of object type for compatibility
data[grouping_columns] = data[grouping_columns].astype(str).fillna('Unknown')

# Ensure numerical columns are numeric for aggregation
data[numerical_columns] = data[numerical_columns].apply(pd.to_numeric, errors='coerce')

# For each numerical column, apply the optimized logic
for col in numerical_columns:
    # 1. For each group of rows with the same 'VehicleType', 'make', 'Model', 
    # calculate the mean of the column (ignoring NaNs)
    column_mean = data.groupby(grouping_columns)[col].transform('mean')
    
    # 2. Fill the missing values with the calculated mean for each group
    data[col] = data[col].fillna(column_mean)

    # 3. If any missing values still exist in the column (due to no group having values),
    # fill with the global mean of the column
    data[col] = data[col].fillna(data[col].mean())

# Verify if the missing values have been filled correctly
print(data.isnull().sum())

data[grouping_columns] = data[grouping_columns].fillna('Unknown')

# For each numerical column, apply the optimized logic
for col in numerical_columns:
    # 1. For each group of rows with the same 'VehicleType', 'make', 'Model', 
    # calculate the mean of the column (ignoring NaNs)
    column_mean = data.groupby(grouping_columns)[col].transform('mean')
    
    # 2. Fill the missing values with the calculated mean for each group
    data[col] = data[col].fillna(column_mean)

    # 3. If any missing values still exist in the column (due to no group having values),
    # fill with the global mean of the column
    data[col] = data[col].fillna(data[col].mean())

# Verify if the missing values have been filled correctly
print(data.isnull().sum())


UnderwrittenCoverID               0
PolicyID                          0
TransactionMonth                  0
IsVATRegistered                   0
Citizenship                       0
LegalType                         0
Title                             0
Language                          0
Bank                              0
AccountType                       0
MaritalStatus                     0
Gender                            0
Country                           0
Province                          0
PostalCode                        0
MainCrestaZone                    0
SubCrestaZone                     0
ItemType                          0
mmcode                          552
VehicleType                       0
RegistrationYear                  0
make                              0
Model                             0
Cylinders                         0
cubiccapacity                     0
kilowatts                         0
bodytype                          0
NumberOfDoors               

In [13]:
# Define the problematic columns
problematic_columns = ['mmcode', 'NumberOfVehiclesInFleet']

# Iterate through the problematic columns
for col in problematic_columns:
    # Calculate the mean for each group and fill missing values with it
    column_mean = data.groupby(grouping_columns)[col].transform('mean')
    data[col] = data[col].fillna(column_mean)

    # Fill any remaining missing values with the global mean
    data[col] = data[col].fillna(data[col].mean())

# Verify again if the missing values have been handled
print(data.isnull().sum())


UnderwrittenCoverID               0
PolicyID                          0
TransactionMonth                  0
IsVATRegistered                   0
Citizenship                       0
LegalType                         0
Title                             0
Language                          0
Bank                              0
AccountType                       0
MaritalStatus                     0
Gender                            0
Country                           0
Province                          0
PostalCode                        0
MainCrestaZone                    0
SubCrestaZone                     0
ItemType                          0
mmcode                            0
VehicleType                       0
RegistrationYear                  0
make                              0
Model                             0
Cylinders                         0
cubiccapacity                     0
kilowatts                         0
bodytype                          0
NumberOfDoors               